In [1]:
#necessary imports
import pandas as pd
import numpy as np

In [2]:
#imports for multi-criteria decision analysis
import skcriteria as skc #!pip install scikit-criteria to use the library
from skcriteria.madm import simple
from skcriteria.preprocessing import invert_objectives, scalers

In [3]:
#read data
town_data=pd.read_csv('college_town_data.csv')

In [4]:
#check data
town_data.head()

,college_towns,population,unemployment_rate,median_income,median_age,cost_of_living_index,median_download_speed,median_upload_speed,median_latency,num_coworking_space,walk_score,bike_score,num_eateries
0,"Auburn, Alabama",74637,2.3,37406,24.1,97.6,192.820,22.160,16.0,2,28,38,240
1,"Fairfield, Alabama",10108,3.8,34556,36.8,76.2,180.565,22.835,17.0,2,30,36,208
2,"Florence, Alabama",39709,3.2,35632,37.1,79.4,171.230,15.560,27.0,2,32,36,151
3,"Homewood, Alabama",26206,1.9,62431,31.5,114.2,144.470,22.190,17.0,4,42,25,177
4,"Huntsville, Alabama",210081,3.1,49060,36.9,90.7,238.980,105.610,11.0,5,24,40,714


In [5]:
#check for null
town_data.isna().sum().sum()

0

In [6]:
#check for duplicates
town_data.duplicated().sum()

0

In [7]:
#create needed columns
town_data['num_eateries_per_capita']=town_data['num_eateries']/town_data['population']
town_data['num_coworking_space_per_capita']=town_data['num_coworking_space']/town_data['population']
town_data['median_income_per_capita']=town_data['median_income']/town_data['population']
town_data['internet_speed_score']=(0.5*town_data['median_download_speed'])+(0.3*town_data['median_upload_speed'])+\
                                (0.2*town_data['median_latency'])
town_data['active_mobility_score']=(0.65*town_data['walk_score'])+(0.35*town_data['bike_score'])

In [9]:
#extract needed columns to use as metric
town_data_metric=town_data[['college_towns','unemployment_rate','cost_of_living_index','num_eateries_per_capita', 
                            'num_coworking_space_per_capita','median_income_per_capita', 'internet_speed_score',
                            'active_mobility_score']]

In [11]:
#see first 5 rows of metric table
town_data_metric.head()

,college_towns,unemployment_rate,cost_of_living_index,num_eateries_per_capita,num_coworking_space_per_capita,median_income_per_capita,internet_speed_score,active_mobility_score
0,"Auburn, Alabama",2.3,97.6,0.003216,0.000027,0.501172,106.258,31.50
1,"Fairfield, Alabama",3.8,76.2,0.020578,0.000198,3.418678,100.533,32.10
2,"Florence, Alabama",3.2,79.4,0.003803,0.000050,0.897328,95.683,33.40
3,"Homewood, Alabama",1.9,114.2,0.006754,0.000153,2.382317,82.292,36.05
4,"Huntsville, Alabama",3.1,90.7,0.003399,0.000024,0.233529,153.373,29.60


#### Multi-Criteria Decision Analysis

In [12]:
#setup for decision matrix object
data=town_data_metric.iloc[:,1:]
objectives=[min,min,max,max,max,max,max]
weights=[0.1,0.2,0.1,0.1,0.15,0.2,0.15]

`The weights used for each attributes:- Unemployment rate: 10%,  Cost of living index: 20%, Number of eateries per capita: 10%, Number of coworking spaces per capita: 10%, Median income per capita: 15%, Speed score: 20%, Active Mobility Score: 15%`

In [14]:
#convert data to numpy array
matrix=np.array(data)
#decision matrix
decision_matrix=skc.mkdm(matrix,objectives,weights,
                        criteria=data.columns,
                        alternatives=town_data_metric['college_towns'].values)
#view decision  matrix
decision_matrix

,unemployment_rate[▼ 0.10],cost_of_living_index[▼ 0.20],num_eateries_per_capita[▲ 0.10],num_coworking_space_per_capita[▲ 0.10],median_income_per_capita[▲ 0.15],internet_speed_score[▲ 0.20],active_mobility_score[▲ 0.15]
"Auburn, Alabama",2.3,97.6,0.003216,0.000027,0.501172,106.2580,31.50
"Fairfield, Alabama",3.8,76.2,0.020578,0.000198,3.418678,100.5330,32.10
"Florence, Alabama",3.2,79.4,0.003803,0.000050,0.897328,95.6830,33.40
"Homewood, Alabama",1.9,114.2,0.006754,0.000153,2.382317,82.2920,36.05
"Huntsville, Alabama",3.1,90.7,0.003399,0.000024,0.233529,153.3730,29.60
...,...,...,...,...,...,...,...
"Platteville, Wisconsin",3.8,81.1,0.018006,0.000085,3.376592,28.9150,46.45
"Stevens Point, Wisconsin",4.1,79.9,0.004310,0.000078,1.556423,117.8740,51.65
"Waukesha, Wisconsin",4.3,100.1,0.002496,0.000028,0.815151,116.7980,36.05
"Whitewater, Wisconsin",4.7,86.6,0.014754,0.000070,2.107384,119.1245,46.45


In [15]:
#transformation to set all objectives to maximize
inverter = invert_objectives.InvertMinimize()
decision_matrix_transformed_objectives = inverter.transform(decision_matrix)
decision_matrix_transformed_objectives

,unemployment_rate[▲ 0.10],cost_of_living_index[▲ 0.20],num_eateries_per_capita[▲ 0.10],num_coworking_space_per_capita[▲ 0.10],median_income_per_capita[▲ 0.15],internet_speed_score[▲ 0.20],active_mobility_score[▲ 0.15]
"Auburn, Alabama",0.434783,0.010246,0.003216,0.000027,0.501172,106.2580,31.50
"Fairfield, Alabama",0.263158,0.013123,0.020578,0.000198,3.418678,100.5330,32.10
"Florence, Alabama",0.312500,0.012594,0.003803,0.000050,0.897328,95.6830,33.40
"Homewood, Alabama",0.526316,0.008757,0.006754,0.000153,2.382317,82.2920,36.05
"Huntsville, Alabama",0.322581,0.011025,0.003399,0.000024,0.233529,153.3730,29.60
...,...,...,...,...,...,...,...
"Platteville, Wisconsin",0.263158,0.012330,0.018006,0.000085,3.376592,28.9150,46.45
"Stevens Point, Wisconsin",0.243902,0.012516,0.004310,0.000078,1.556423,117.8740,51.65
"Waukesha, Wisconsin",0.232558,0.009990,0.002496,0.000028,0.815151,116.7980,36.05
"Whitewater, Wisconsin",0.212766,0.011547,0.014754,0.000070,2.107384,119.1245,46.45


In [16]:
#apply sumscaler to bring all criteria to same scale
scaler = scalers.SumScaler(target="both")
decision_matrix_scaled = scaler.transform(decision_matrix_transformed_objectives)
decision_matrix_scaled

,unemployment_rate[▲ 0.10],cost_of_living_index[▲ 0.20],num_eateries_per_capita[▲ 0.10],num_coworking_space_per_capita[▲ 0.10],median_income_per_capita[▲ 0.15],internet_speed_score[▲ 0.20],active_mobility_score[▲ 0.15]
"Auburn, Alabama",0.002956,0.001267,0.000249,0.000157,0.000149,0.001270,0.000992
"Fairfield, Alabama",0.001789,0.001623,0.001591,0.001156,0.001016,0.001202,0.001011
"Florence, Alabama",0.002125,0.001557,0.000294,0.000294,0.000267,0.001144,0.001052
"Homewood, Alabama",0.003578,0.001083,0.000522,0.000892,0.000708,0.000984,0.001135
"Huntsville, Alabama",0.002193,0.001363,0.000263,0.000139,0.000069,0.001834,0.000932
...,...,...,...,...,...,...,...
"Platteville, Wisconsin",0.001789,0.001525,0.001392,0.000496,0.001003,0.000346,0.001463
"Stevens Point, Wisconsin",0.001658,0.001548,0.000333,0.000454,0.000462,0.001409,0.001627
"Waukesha, Wisconsin",0.001581,0.001235,0.000193,0.000164,0.000242,0.001396,0.001135
"Whitewater, Wisconsin",0.001446,0.001428,0.001141,0.000407,0.000626,0.001424,0.001463


#### Ranking Based on Weighted sum

```The Weighted Sum Method is a popular multi-criteria decision analysis technique used to evaluate and rank alternatives based on multiple criteria. It works by assigning weights to each criterion, which represents its relative importance, and then multiplying each criterion by its weight. The products are then summed up to produce an overall score for each alternative. The alternative with the highest score is selected as the best option. The Weighted Sum Method is simple to implement and can be used in a variety of decision-making contexts```

In [17]:
#compute ranking using Weighted Sum Model
decision = simple.WeightedSumModel()
ws_rank = decision.evaluate(decision_matrix_scaled)  # we use the tansformed version of the data
#see rank
ws_rank

Alternatives,"Auburn, Alabama","Fairfield, Alabama","Florence, Alabama","Homewood, Alabama","Huntsville, Alabama","Jacksonville, Alabama","Livingston, Alabama","Mobile, Alabama","Montevallo, Alabama","Montgomery, Alabama","Troy, Alabama","Tuscaloosa, Alabama","Tuskegee, Alabama","Anchorage, Alaska","Fairbanks, Alaska","Juneau, Alaska","Ketchikan, Alaska","Sitka, Alaska","Chandler, Arizona","Flagstaff, Arizona","Gilbert, Arizona","Glendale, Arizona","Lake Havasu City, Arizona","Mesa, Arizona","Peoria, Arizona","Prescott, Arizona","Scottsdale, Arizona","Tempe, Arizona","Tucson, Arizona","Arkadelphia, Arkansas","Conway, Arkansas","Fayetteville, Arkansas","Fort Smith, Arkansas","Jonesboro, Arkansas","Magnolia, Arkansas","Monticello, Arkansas","Russellville, Arkansas","Searcy, Arkansas","Angwin, California","Arcata, California","Azusa, California","Berkeley, California","Chico, California","Claremont, California","Cotati, California","Davis, California","Irvine, California","Isla Vista, California","La Verne, California","Loma Linda, California","Merced, California","Orange, California","Pasadena, California","Pomona, California","Redlands, California","Riverside, California","Rocklin, California","San Luis Obispo, California","Santa Barbara, California","Santa Cruz, California","Seaside, California","Stanford, California","Turlock, California","University District, California","Whittier, California","Alamosa, Colorado","Aurora, Colorado","Boulder, Colorado","Colorado Springs, Colorado","Durango, Colorado","Fort Collins, Colorado","Golden, Colorado","Grand Junction, Colorado","Greeley, Colorado","Gunnison, Colorado","Lakewood, Colorado","Pueblo, Colorado","Bridgeport, Connecticut","Danbury, Connecticut","East Hartford, Connecticut","Fairfield, Connecticut","Farmington, Connecticut","Hamden, Connecticut","Hartford, Connecticut","Middletown, Connecticut","New Britain, Connecticut","New Haven, Connecticut","New London, Connecticut","North Haven, Connecticut","Southington, Connecticut","Stamford, Connecticut","Storrs, Connecticut","Waterbury, Connecticut","Willimantic, Connecticut","West Hartford, Connecticut","West Haven, Connecticut","Dover, Delaware","Newark, Delaware","New Castle, Delaware","Wilmington, Delaware","Boca Raton, Florida","Bradenton, Florida","Brandon, Florida","Cape Coral, Florida","Coral Gables, Florida","Coral Springs, Florida","Davie, Florida","DeLand, Florida","Estero, Florida","Fort Lauderdale, Florida","Gainesville, Florida","Gulfport, Florida","Hialeah, Florida","Hollywood, Florida","Kendall, Florida","Lakeland, Florida","Miami Gardens, Florida","Miami Shores, Florida","Miramar, Florida","Pembroke Pines, Florida","Pensacola, Florida","Port St. Lucie, Florida","Sarasota, Florida","St. Leo, Florida","St. Petersburg, Florida","Sunrise, Florida","Tallahassee, Florida","West Palm Beach, Florida","Albany, Georgia","Athens, Georgia","Augusta, Georgia","Blue Ridge, Georgia","Brunswick, Georgia","Carrollton, Georgia","Columbus, Georgia","Dahlonega, Georgia","Demorest, Georgia","Fort Valley, Georgia","Glennville, Georgia","Kennesaw, Georgia","Lawrenceville, Georgia","Madison, Georgia","Macon, Georgia","Milledgeville, Georgia","Mount Vernon, Georgia","Oxford, Georgia","Rome, Georgia","Savannah, Georgia","Statesboro, Georgia","Swainsboro, Georgia","Valdosta, Georgia","Vidalia, Georgia","Waleska, Georgia","Watkinsville, Georgia","Young Harris, Georgia","Laie, Hawaii","Boise, Idaho","Caldwell, Idaho","Lewiston, Idaho","Moscow, Idaho","Nampa, Idaho","Pocatello, Idaho","Rexburg, Idaho","Aurora, Illinois","Bloomington, Illinois","Bourbonnais, Illinois","Carbondale, Illinois","Charleston, Illinois","Deerfield, Illinois","DeKalb, Illinois","East Peoria, Illinois","Edwardsville, Illinois","Elgin, Illinois","Elmhurst, Illinois","Evanston, Illinois","Joliet, Illinois","Lebanon, Illinois","Macomb, Illinois","Naperville, Illinois","Normal, Illinois","Peoria, Illinois","Rockford, Illinois","Springfield, Illinois","University Park, Illi

In [18]:
#see few rank scores
ws_rank.e_.score[0:15]

array([0.0010147 , 0.00132258, 0.0010093 , 0.00118902, 0.00104911,
       0.00121196, 0.00192685, 0.00084636, 0.00163854, 0.00085792,
       0.00100862, 0.00093078, 0.00128649, 0.00073968, 0.00093651])

In [19]:
#make new dataframe with metrics and scores
ranking_df=pd.DataFrame({'town':ws_rank.alternatives,'Rank':ws_rank.rank_,
              'Ranking Score':ws_rank.e_.score})

#concat with town_data_metric
college_towns_ranking_ws=pd.concat([town_data_metric,ranking_df],axis=1)

#see first 5 rows
college_towns_ranking_ws.head()

,college_towns,unemployment_rate,cost_of_living_index,num_eateries_per_capita,num_coworking_space_per_capita,median_income_per_capita,internet_speed_score,active_mobility_score,town,Rank,Ranking Score
0,"Auburn, Alabama",2.3,97.6,0.003216,0.000027,0.501172,106.258,31.50,"Auburn, Alabama",398,0.001015
1,"Fairfield, Alabama",3.8,76.2,0.020578,0.000198,3.418678,100.533,32.10,"Fairfield, Alabama",196,0.001323
2,"Florence, Alabama",3.2,79.4,0.003803,0.000050,0.897328,95.683,33.40,"Florence, Alabama",404,0.001009
3,"Homewood, Alabama",1.9,114.2,0.006754,0.000153,2.382317,82.292,36.05,"Homewood, Alabama",254,0.001189
4,"Huntsville, Alabama",3.1,90.7,0.003399,0.000024,0.233529,153.373,29.60,"Huntsville, Alabama",348,0.001049


In [21]:
#validate if towns are the same in each row
(college_towns_ranking_ws['college_towns']==college_towns_ranking_ws['town']).sum()

738

In [22]:
#drop town column and then sort
college_towns_ranking_ws.drop('town',axis=1,inplace=True)

In [23]:
#see rank
college_towns_ranking_ws=college_towns_ranking_ws.sort_values(by='Rank')
college_towns_ranking_ws

,college_towns,unemployment_rate,cost_of_living_index,num_eateries_per_capita,num_coworking_space_per_capita,median_income_per_capita,internet_speed_score,active_mobility_score,Rank,Ranking Score
450,"Montreat, North Carolina",4.3,121.7,0.865889,0.005831,178.571429,123.5050,31.05,1,0.018865
560,"Lower Merion Township, Pennsylvania",7.9,77.8,0.253401,0.003401,72.988095,104.9530,57.80,2,0.008128
449,"Misenheimer, North Carolina",4.1,76.7,0.423077,0.002849,49.857550,123.5050,31.05,3,0.008089
123,"St. Leo, Florida",5.0,116.7,0.285826,0.001566,72.435395,120.6205,39.15,4,0.007175
590,"Due West, South Carolina",5.4,75.2,0.384972,0.001855,44.179035,112.6820,32.15,5,0.006906
...,...,...,...,...,...,...,...,...,...,...
55,"Riverside, California",7.1,134.1,0.002334,0.000010,0.178141,114.8350,45.10,734,0.000799
458,"Wilson, North Carolina",7.2,78.5,0.002838,0.000021,0.793714,67.3760,33.10,735,0.000796
116,"Miami Gardens, Florida",12.7,114.4,0.001307,0.000009,0.354010,111.2230,39.75,736,0.000755
13,"Anchorage, Alaska",6.6,116.0,0.002383,0.000003,0.267039,87.8270,38.35,737,0.000740
